In [1]:
import sys
sys.path.append('../')

import joblib
import pandas as pd
from scipy.stats import norm

import covasim.covasim as cv
import covasim.covasim.utils as cvu
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
import sciris as sc
import numpy as np

Covasim 3.1.3 (2022-07-19) — © 2020-2022 by IDM


In [2]:
class norm_random_masking(cv.Intervention):
  def __init__(self,mask_eff=None,maskprob_ub=None,maskprob_lb=None,mean=None,std=None,*args,**kwargs):
    super().__init__(**kwargs)
    self.mask_eff     = mask_eff
    self.maskprob_ub  = maskprob_ub
    self.maskprob_lb  = maskprob_lb
    self.mean         = mean
    self.std          = std
    self.t            = []
    self.num_masking  = []
    self.mask_avg_lst = []
    return

  def initialize(self,sim):
    super().initialize()
    self.pop = len(sim.people)
    self.pop        = len(sim.people)
    self.child      = np.logical_and(sim.people.age > 2, sim.people.age <= 9)
    self.adolescent = np.logical_and(sim.people.age > 9, sim.people.age <= 19)
    self.adult      = np.logical_and(sim.people.age > 19, sim.people.age <= 69)
    self.senior     = np.logical_and(sim.people.age > 69, sim.people.age <= 79 )
    self.supsenior  = sim.people.age > 79
    return

  def apply(self,sim):
    ppl                = sim.people
    ppl.rel_sus[self.child]        = 0.34
    ppl.rel_sus[self.adolescent]   = 0.67
    ppl.rel_sus[self.adult]        = 1.00
    ppl.rel_sus[self.senior]       = 1.24
    ppl.rel_sus[self.supsenior]    = 1.47
    self.norm          = norm.rvs(loc=self.mean,scale=self.std,size=self.pop)
    self.num_dead      = ppl.dead.sum()
    self.num_diagnosed = (ppl.diagnosed & ppl.infectious).sum()
    self.x             = self.num_dead + self.num_diagnosed
    self.p             = np.exp(0.001 + (self.norm*(self.x/self.pop))-0.001*sim.t)
    self.p             = (self.p/(1+self.p))-0.5
    self.p             = np.clip(self.p,self.maskprob_lb,self.maskprob_ub)
    self.masking       = np.random.binomial(1,p=self.p,size=self.pop)
    ppl.rel_sus        = np.where(self.masking,ppl.rel_sus*self.mask_eff,ppl.rel_sus)
    self.num_masking.append(np.sum(self.masking))
    self.t.append(sim.t)
    self.total_t = self.t[-1]
    self.total_masked  = sum(self.num_masking)
    self.mask_avg      = self.total_masked/self.total_t+1
    self.mask_avg_lst.append(self.mask_avg)
    self.final_mask_avg = self.mask_avg_lst[-1]
    return

  def get_mask_avg(self,sim): # when sim finishes it'll print the average number of people masking over the entire sim 
    if len(self.t) > sim.pars['n_days']:
      print(self.final_mask_avg)
    else:
      None
    return 


  def plot(self):
    plt.plot(self.t,self.num_masking)
    plt.xlabel('Day')
    plt.ylabel('# of Agents Masking')
    plt.title('# of Agents Masking Over Time')
    plt.show()
    return


In [3]:
tp = cv.test_prob(symp_prob=0.1,asymp_prob=0.001,symp_quar_prob=0.3,asymp_quar_prob=0.3,quar_policy='daily')
rm = norm_random_masking(mask_eff=0.6,maskprob_ub=0.75,maskprob_lb=0.00,mean=75,std=50)

In [ ]:
sim = cv.Sim(n_days=90,interventions=[tp,rm],label='masking')
n_runs = 5
msim = cv.MultiSim(sim,n_runs=n_runs)

msim.run()
for i in range(n_runs): 
  mask_avg = msim.sims[i].get_intervention(norm_random_masking)
  mask_avg.get_mask_avg(sim)


In [4]:
sim = cv.Sim(n_days=90,interventions=[tp,rm],label='masking')
sim1 = cv.Sim(interventions=tp,label='no masking',n_days=90)
msim = cv.MultiSim([sim,sim1])

msim.run()
msim.plot()
num_mask = msim.sims[0].get_intervention(norm_random_masking)
num_mask.plot()

NameError: Task 0 failed: set die=False to keep going instead; see above for error details